In [2]:
import findspark
findspark.init("/spark")

import os
basepath =  os.path.join('c:/', 'Users', 'gonplaza', 'Documents', 'gg', 'serie-notas')
os.chdir(basepath)

from z_commons.commons import _split_s3_path

from pyspark.sql import SparkSession

spark = (
    SparkSession
        .builder
        .appName('load_data')
        .getOrCreate()
        )

spark

ModuleNotFoundError: No module named 'z_commons.commons'

In [2]:
# DataFrames

# Estructura de datos especializada para trabajar con datos estructurados.

# Funciona con *lazy-evaluation*. O sea, no ejecuta los computos al ser definidos, 
# sino al ser necesarios para generar un resultado.

# Es similar a la dataframe de R o Pandas. O sea, tiene columnas con nombre, 
# donde cada columna puede contener sólo 1 tipo de dato.

# Una pyspark DataFrame se crea habitualmente a partir de una lista de lista, 
# de un diccionario, pyspark.sql.Rows, una pandas.DataFrame o un RDD.

In [3]:
# from a list of dicts
data = [{"Category": 'A', "ID": 1, "Value": 121.44, "Truth": True},
        {"Category": 'B', "ID": 2, "Value": 300.01, "Truth": False},
        {"Category": 'C', "ID": 3, "Value": 10.99, "Truth": None},
        {"Category": 'E', "ID": 4, "Value": 33.87, "Truth": True}
        ]

df = spark.createDataFrame(data)
df.show()

+--------+---+-----+------+
|Category| ID|Truth| Value|
+--------+---+-----+------+
|       A|  1| true|121.44|
|       B|  2|false|300.01|
|       C|  3| null| 10.99|
|       E|  4| true| 33.87|
+--------+---+-----+------+



In [4]:
# Creación de DataFrame a partir de Pandas.dataFrame
import pandas as pd
from datetime import datetime, date

pandas_df = pd.DataFrame({
    'a': [1, 2, 3],
    'b': [2., 3., 4.],
    'c': ['string1', 'string2', 'string3'],
    'd': [date(2000, 1, 1), date(2000, 2, 1), date(2000, 3, 1)],
    'e': [datetime(2000, 1, 1, 12, 0), datetime(2000, 1, 2, 12, 0), datetime(2000, 1, 3, 12, 0)]
})

df = spark.createDataFrame(pandas_df)
df.show()

c:\spark\python\pyspark\sql\pandas\conversion.py:479: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:


+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
|  2|3.0|string2|2000-02-01|2000-01-02 12:00:00|
|  3|4.0|string3|2000-03-01|2000-01-03 12:00:00|
+---+---+-------+----------+-------------------+



In [5]:
# from list of tuples, with eplicit schema.
from datetime import date, datetime

df = spark.createDataFrame([
    (1, 2., 'string1', date(2000, 1, 1), datetime(2000, 1, 1, 12, 0)),
    (2, 3., 'string2', date(2000, 2, 1), datetime(2000, 1, 2, 12, 0)),
    (3, 4., 'string3', date(2000, 3, 1), datetime(2000, 1, 3, 12, 0))
], schema='a long, b double, c string, d date, e timestamp')

df.show()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
|  2|3.0|string2|2000-02-01|2000-01-02 12:00:00|
|  3|4.0|string3|2000-03-01|2000-01-03 12:00:00|
+---+---+-------+----------+-------------------+



In [6]:
type(df)

pyspark.sql.dataframe.DataFrame

In [7]:
df.printSchema()

root
 |-- a: long (nullable = true)
 |-- b: double (nullable = true)
 |-- c: string (nullable = true)
 |-- d: date (nullable = true)
 |-- e: timestamp (nullable = true)



In [8]:
df.show(1)

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
+---+---+-------+----------+-------------------+
only showing top 1 row



In [9]:
df.show(1, vertical=True)

-RECORD 0------------------
 a   | 1                   
 b   | 2.0                 
 c   | string1             
 d   | 2000-01-01          
 e   | 2000-01-01 12:00:00 
only showing top 1 row



In [10]:
df.columns

['a', 'b', 'c', 'd', 'e']

In [14]:

_split_s3_path("s3://mid-ds-reports-rt-idx-analysis/gplaza/ds-nse-dev/data/relevant_geo_data/country=MX/part-00000-e1603161-7a4c-40d3-ba9a-7a0ed733e5ad-c000.snappy.parquet")

('mid-ds-reports-rt-idx-analysis',
 'gplaza/ds-nse-dev/data/relevant_geo_data/country=MX/part-00000-e1603161-7a4c-40d3-ba9a-7a0ed733e5ad-c000.snappy.parquet')

In [19]:
import awswrangler as wr
import boto3

In [20]:
df = wr.s3.read_parquet(path)
df.head(2)

NoCredentialsError: Unable to locate credentials

In [ ]:
# 1. De una carpeta , elegir un file y quedarese con el.
# 2. bajarlo a un tmp.
# 3 levantarlo con pandas
# 5. pasarlo a dataframe

In [17]:
df = spark.read.load(path)

Py4JJavaError: An error occurred while calling o98.load.
: org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "s3"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3443)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:724)
	at scala.collection.immutable.List.map(List.scala:293)
	at org.apache.spark.sql.execution.datasources.DataSource$.checkAndGlobPathIfNecessary(DataSource.scala:722)
	at org.apache.spark.sql.execution.datasources.DataSource.checkAndGlobPathIfNe